# Feature extraction

Steps:
    
    - Detrending IIR Filter, Subsampling and Buterworth filter
    
    - Time domain features on each measurement (fault (number), severity (number), rpm, feature vector ...)
    
    - Save on disk - Load features from disk
    
    - (Later FD, TFD features)
    
    - PCM, DPCM, Features size

In [ ]:
from zipfile import ZipFile
from itertools import pairwise
import pandas as pd
import numpy as np

from scipy.stats import skew, kurtosis, entropy
from scipy.signal import welch, windows, find_peaks
from scipy.interpolate import interp1d #, CubicSpline, PchipInterpolator
from scipy.fft import rfft

import seaborn as sb
import matplotlib.pyplot as plt
import mafaulda as src

MAFAULDA_PATH = '../../datasets/MAFAULDA.zip'

## Preprocessing

- DC Removal
- ANC - Noise removal
- Downsampling + LowPass

In [ ]:
# https://medium.com/analytics-vidhya/how-to-filter-noise-with-a-low-pass-filter-python-885223e5e9b7
# Low-pass filter 10 kHz (Butterworth)

# Downsampling factor: 50 kHz / 10 kHz = 5
# src.axis_spectrograms(misalign_sub)  (čistý subsampling vs. low-pass)

## Time domain features

In [ ]:
dataset = ZipFile(MAFAULDA_PATH)

def parse_filename(filename):
    fault = filename.split('/')
    if fault[0].strip() in ('overhang', 'underhang'):
        fault = f'{fault[0]}-{fault[1]}'
        severity = fault[2]
        seq = fault[3]
    elif fault[0].strip() == 'normal':
        fault, severity, seq = fault[0], '', fault[1]
    else:
        fault, severity, seq = fault

    return fault, severity, seq

def features_time_domain(zip_file, filename):
    print(f'Processing: {filename}')
    
    ts = src.csv_import(zip_file, filename)
    fault, severity, seq = parse_filename(filename)
    rpm = ts['rpm'].mean()
    columns = ['ax', 'ay', 'az', 'bx', 'by', 'bz', 'mic']
    
    feature_vector = [
        ts[columns].mean().rename('mean'),
        ts[columns].std().rename('std'),
        ts[columns].apply(lambda x: skew(x)).rename('skew'),
        ts[columns].apply(lambda x: kurtosis(x)).rename('kurt'),
        ts[columns].apply(src.rms).rename('rms'),
        ts[columns].apply(lambda x: np.max(x) - np.min(x)).rename('pp'),
        ts[columns].apply(lambda x: np.max(np.absolute(x)) / src.rms(x)).rename('crest'),
        ts[columns].apply(lambda x: np.max(np.absolute(x)) / np.mean(np.sqrt(np.absolute(x))) ** 2).rename('margin'),
        ts[columns].apply(lambda x: np.max(np.absolute(x)) / np.mean(np.absolute(x))).rename('impulse'),
        ts[columns].apply(lambda x: src.rms(x) / np.mean(np.absolute(x))).rename('shape'),
        ts[columns].max().rename('max'),
    ]
    return (
        pd.concat(feature_vector, axis=1)
        .assign(fault=fault, severity=severity, seq=seq, rpm=rpm)
        .reset_index()
        .rename(columns={'index': 'axis'})
    )
    

files = src.get_mafaulda_files(dataset) # [:10]  # TODO: all
td_features = src.import_files(dataset, files, features_time_domain)
td_features

In [ ]:
# td_features.to_csv('td_features_no_filter.csv', index=False)

### Correlations in features

In [ ]:
features = pd.read_csv('td_features_no_filter.csv')
td_columns = ['mean', 'std', 'skew', 'kurt', 'rms', 'pp', 'crest', 'margin', 'impulse', 'shape']
features[td_columns].corr()

In [ ]:
sb.heatmap(features[td_columns].corr(), cmap="YlGnBu", annot=True)

In [ ]:
ax = features[td_columns].var().plot(kind='barh', xlabel='Variance', ylabel='Feature')

## Frequency domain features
Read also: https://librosa.org/doc/0.10.1/feature.html

In [ ]:
OVERLAP = 0.5
WINDOW_SIZES = (2**8, 2**10, 2**12, 2**14, 2**16)

for w in WINDOW_SIZES:
    src.resolution_calc(src.FS_HZ, w)
    print()

In [ ]:
def find_harmonics(f: np.array, Pxx: np.array) -> (np.array, np.array):
    threshold = Pxx.mean() +  2*np.std(Pxx)
    peaks, _ = find_peaks(Pxx)
    f_harmonics = f[peaks]
    y_harmonics = Pxx[peaks]

    cond = y_harmonics >= threshold
    loc_harmonics = peaks[cond]
    f_harmonics = f_harmonics[cond]
    return loc_harmonics, f_harmonics
    

def envelope_signal(f: np.array, Pxx: np.array) -> np.array:
    peaks, _ = find_peaks(Pxx)
    envelope = interp1d(f[peaks], Pxx[peaks], kind='quadratic', fill_value='extrapolate')
    y_env = envelope(f)
    y_env[y_env < 0] = 0
    return y_env


def spectral_transform(dataset: pd.DataFrame, axis: str, window: int) -> (np.array, np.array):
    OVERLAP = 0.5
    STEP = int(window * OVERLAP)
    v = dataset[axis].to_numpy()
    f, Pxx = welch(
        v, fs=src.FS_HZ, window='hann',
        nperseg=window, noverlap=STEP,
        scaling='spectrum', average='mean', detrend='constant',
        return_onesided=True
    )
    return f, Pxx


def energy(Pxx: np.array) -> float:
    return np.sum(Pxx**2)


def spectral_roll_off_frequency(f: np.array, Pxx: np.array, percentage: float) -> float:
    # Roll-off: Cumulative sum of energy in spectral bins and find index in f array
    # 85% of total energy below this frequency
    return f[np.argmax(np.cumsum(Pxx**2) >= percentage * energy(Pxx))]


def temporal_variation(dataset: pd.DataFrame, axis: str, window: int) -> list:
    # Temporal variation of succesive spectra (stationarity)
    OVERLAP = 0.5
    STEP = int(window * OVERLAP)
    v = dataset[axis].to_numpy()
    spectra = [
        np.absolute(rfft(v[i:i+window] * windows.hann(window)))
        for i in range(0, len(v) - window, STEP)
    ]
    # f = [i * (src.FS_HZ / window) for i in range(window // 2 + 1)]
    fluxes = [
        1 - np.corrcoef(psd1, psd2) for psd1, psd2 in pairwise(spectra)
    ]
    return fluxes


def features_frequency_domain(zip_file: ZipFile, filename: str):
    # Calculate FFT with Welch method in 5 different Hann window sizes
    print(f'Processing: {filename}')
    
    ts = src.csv_import(zip_file, filename)
    fault, severity, seq = parse_filename(filename)
    rpm = ts['rpm'].mean()
    columns = ['ax', 'ay', 'az', 'bx', 'by', 'bz', 'mic']
    result = []
    
    for window in WINDOW_SIZES:
        for col in columns:
            f, Pxx = spectral_transform(ts, col, window)
            
            fluxes = temporal_variation(ts, col, window)
            # TODO: harmonic part energy, noise part energy
            noisiness = 0
            inharmonicity = 0
            # TODO: energies of each bin
            envelope_spectrum = envelope_signal(f, Pxx)
            loc_harmonics, _ = find_harmonics(f, Pxx)
            # TODO: implement harmonics finder according to paper
            
            
            result.append({
                'fft_window_length': window,
                'fault': fault,
                'severity': severity,
                'seq': seq,
                'rpm': rpm,
                'axis': col,
                'centroid': np.average(f, weights=Pxx),
                'std': np.std(Pxx),
                'skew': skew(Pxx),
                'kurt': kurtosis(Pxx),
                'roll-off': spectral_roll_off_frequency(f, Pxx, 0.85),
                'flux_mean': np.mean(fluxes),
                'flux_std': np.std(fluxes),
                'hdev': np.mean(envelope_spectrum[loc_harmonics] - Pxx[loc_harmonics]),
                'noisiness': noisiness,
                'inharmonicity': inharmonicity,
                'energy': energy(Pxx),
                'entropy': entropy(Pxx / np.sum(Pxx)),
                'negentropy': -entropy((envelope_spectrum ** 2) / np.mean(envelope_spectrum ** 2))
            })

    return pd.DataFrame(result)

### Extract frequency domain features

In [ ]:
dataset = ZipFile(MAFAULDA_PATH)
files = src.get_mafaulda_files(dataset)[:10]  # TODO: all
fd_features = src.import_files(dataset, files, features_frequency_domain)
fd_features

In [ ]:
fd_features.to_csv('fd_features_no_filter.csv')

### Experiment to find spectral envelope

In [ ]:
# TODO: calculate size of signal in PCM and DPCM (biggest number for differential), DPCM with len(hamming code) >= entropy in time domain

# Experiment to find spectral envelope
def plot_spectral_envelope(dataset, file, axis):
    ts = src.csv_import(dataset, file)
    f, Pxx = spectral_transform(ts, axis, 1024)
    y_env = envelope_signal(f, Pxx)
    # print(find_harmonics(f, Pxx))
    
    fig, ax = plt.subplots(1, 1, figsize=(20, 5))
    ax.plot(f, Pxx)
    ax.scatter(f[peaks], Pxx[peaks], color='red')
    ax.plot(f, y_env)

plot_spectral_envelope(ZipFile(MAFAULDA_PATH), 'vertical-misalignment/1.78mm/51.8144.csv', 'ax')

## Time-frequency domain features

#### Features:
- Energy
- Entropy

#### Transforms:
- Discrete wavelet transform
- Wavelet packet decompostion (Fejér-Korovkin wavelet)
- Empirical wavelet transform

In [ ]:
import pywt
import ewtpy
import warnings

dataset = ZipFile(MAFAULDA_PATH)
ts = src.csv_import(dataset, 'vertical-misalignment/1.78mm/51.8144.csv')

### Multilevel 1D Discrete Wavelet Transform
https://www.mathworks.com/help/wavelet/gs/choose-a-wavelet.html

In [ ]:
axis = 'ax'
wavelet = 'db8'
result = pywt.wavedec(ts[axis], wavelet, mode='symmetric', level=3)
print(len(ts[axis]))
print([len(x) for x in result], '\n', result)

### Wavelet packet decomposition

In [ ]:
wp = pywt.WaveletPacket(data=ts['ax'], wavelet='db1', mode='symmetric')

# Collect nodes based on frequency
level = 3
print([node.path for node in wp.get_level(level, 'freq')])
print([len(node.data) for node in wp.get_level(level, 'freq')])

#### Energy in partitioned regions
- store multiple levels to compare = {3, 6, 9}
- multiple wavelets = {db1, ...}

In [ ]:
levels = 3
print('Energy')
energies = [
    energy(node.data) for node in wp.get_level(levels, 'freq')
]
total_energy = np.sum(energies)
print(energies)
print('Total Energy')
print(total_energy)

print('Energy ratio')
energy_ratios = [
    energy(node.data) / total_energy 
    for node in wp.get_level(levels, 'freq')
]
print(energy_ratios)

print('Entropy')
print([entropy(node.data**2) for node in wp.get_level(levels, 'freq')])

In [ ]:
pywt.families()
pywt.wavelist()
w = pywt.Wavelet('db8')
print(w)

### Empirical Wavelet transform

In [ ]:
warnings.simplefilter(action='ignore', category=FutureWarning)

def ewt_transform(dataset: pd.DataFrame, axis: str, scales: int):
    ewt, mfb, boundaries = ewtpy.EWT1D(
        dataset[axis], N=scales,
        log=0, detect='locmax', completion=0, 
        reg='average', lengthFilter=10, sigmaFilter=5
    )
    return ewt, mfb, boundaries
    

ewt, mfb, boundaries = ewt_transform(ts, 'ax', 3)
#files = src.get_mafaulda_files(dataset)[:10]  # TODO: all
#fd_features = src.import_files(dataset, files, features_frequency_domain)
#fd_features
ewt